In [1]:
%load_ext Cython

In [2]:
import numpy as np

In [211]:
# create some random data:
# we want radial data to be of length 
r_len = 6040
# and we want 
z_len = 1801
# different independend slices in z direction.

dtype = np.float
T = np.random.rand(r_len * z_len).astype(dtype).reshape(z_len, r_len)

In [86]:
T

array([[0.58460506, 0.51979563, 0.87702632, 0.06033861, 0.28352344,
        0.84536253],
       [0.8667614 , 0.87796999, 0.07486281, 0.37233445, 0.02236128,
        0.57938297]])

In [79]:
# definie output

#      z
#  |------| 
#  |------| y
#  |------|
#

output = np.zeros(r_len * z_len, dtype=dtype).reshape(r_len, z_len)

In [112]:
# start with numpy and python
for yy in range(r_len):
    r_vec = np.arange( yy, r_len , 1) + 0.5
    # + 0.5 intervall middle (dr)
    # construct K_y
    r_vec = r_vec**2
    r_vec = r_vec - (yy)**2
    r_vec = np.sqrt(r_vec)
    # matrix * vector 
    out = np.dot(T[:, yy:],r_vec)
    output[yy,:] = out 

In [113]:
output

array([[9.40109626, 6.52789535],
       [8.6090017 , 5.63358245],
       [6.96296254, 4.24033906],
       [4.95667219, 3.41706099],
       [3.7756721 , 2.23322169],
       [1.93696889, 1.32753315]])

In [114]:
def rotation_np(input_data):
    z_len, r_len = input_data.shape
    # deifine output:
    #
    #      z
    #  |------| 
    #  |------| y
    #  |------|
    #
    output = np.zeros(r_len * z_len, dtype=dtype).reshape(r_len, z_len)
    for yy in range(r_len):
        r_vec = np.arange( yy, r_len , 1) + 0.5
        # + 0.5 intervall middle (dr)
        # construct K_y
        r_vec = r_vec**2
        r_vec = r_vec - (yy)**2
        r_vec = np.sqrt(r_vec)**(-1)
        # matrix * vector 
        out = np.dot(T[:, yy:],r_vec)
        output[yy,:] = out 
    return output

In [115]:
rotation_np(T)

array([[2.10049816, 2.56547405],
       [1.08660308, 1.04118628],
       [0.8410228 , 0.29816988],
       [0.30138507, 0.33888665],
       [0.36147107, 0.16432911],
       [0.36894646, 0.25286346]])

In [205]:
%%cython -a
cimport cython
cimport numpy as np
import  numpy as np
from libc.math cimport sqrt
from scipy.linalg.cython_blas cimport dgemv
@cython.boundscheck(False)
@cython.wraparound(False)

cdef int _rotation(double [:,::1] input_data, double [:,::1] output_data,
                  double [:] coeff):
    cdef Py_ssize_t r_len, s_len
    r_len = input_data.shape[1]
    s_len = input_data.shape[0]
    cdef Py_ssize_t yy, ii

    cdef char* trans = 't'
    cdef int n = s_len 
    cdef int m
    cdef int lda = r_len
    cdef double alpha = 1
    cdef double beta = 0
    cdef int incx = 1
    cdef int incy = 1
    cdef double* A_start
    cdef double* output_start
    cdef double* coeff_start = &coeff[0]
    for yy in range(r_len):
        m = r_len - yy
        for ii in range (m):
            coeff[ii] = (sqrt((yy + ii + 0.5)**2 - yy**2))**(-1)
        #
        A_start = &input_data[0,yy]
        output_start = &output_data[yy,0]
        #
  
        dgemv(trans, &m, &n, &alpha, A_start, &lda, coeff_start,
                         &incx, &beta, output_start, &incy)
    return 0
        
def rotation(input_data):
    try:
        input_data.flags
    except AttributeError:
        print('input_data should be a numpy array!')
        raise
    if input_data.flags['F_CONTIGUOUS']:
        print('input_data should be stored in a raw major, C contigous.')
    elif not input_data.flags['C_CONTIGUOUS']:
        print('input_data as to be a contigous array.')
    
    output = np.zeros(input_data.size, dtype=np.float64, order='C').reshape(
                    input_data.shape[1], input_data.shape[0])
    coeff = np.empty(input_data.shape[1], dtype=np.float64, order='C')
    
    cdef double [:,::1] input_data_view = input_data
    cdef double [:,::1] output_view = output
    cdef double [:] coeff_view = coeff
    _rotation(input_data_view, output_view, coeff_view)
    return output

In [212]:
%timeit rotation(T)

16.1 s ± 457 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [214]:
rotation_np(T)

KeyboardInterrupt: 

In [146]:
T

array([[0.58460506, 0.51979563, 0.87702632, 0.06033861, 0.28352344,
        0.84536253],
       [0.8667614 , 0.87796999, 0.07486281, 0.37233445, 0.02236128,
        0.57938297]])

In [163]:
T[1,2]

0.0748628099257106